In [1]:
# change directory to use "mapper" package used in MOLI for preprocessing steps

import os

print(os.getcwd())
os.chdir("/Volumes/Expansion/Thesis Work/Codes/")
print(os.getcwd())

/Volumes/Expansion/Files
/Volumes/Expansion/Thesis Work/Codes


In [2]:
# import libraries

from __future__ import print_function
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import zscore
import seaborn as sns
import sys
import xlrd
from mapper import expand, parse_mapping_table, apply_mappers
%matplotlib inline
import warnings

# suppress all warnings
warnings.filterwarnings("ignore")

In [15]:
# define directories

gene_id = "ENTREZID"
raw_data_dir = "/Volumes/Expansion/Thesis Work/Datasets/PDX/Expression/"
preprocessing_data_dir = "/Volumes/Expansion/Thesis Work/Results/preprocessed_results2/"
root_dir = "/Volumes/Expansion/Thesis Work/Supplementary Files/"
tcga_tmp_dir = "/Volumes/Expansion/Thesis Work/Datasets/TCGA/Expression/expression__2016_01_28"

# PDX 

- Gene symbols are converted to Gene IDs 
- gene expression profiles (FPKM) are converted to log2(TPM+1)

In [4]:
# read PDX expression dataset (PDX)

exprs  = pd.read_excel(raw_data_dir+"nm.3954-S2.xlsx","RNAseq_fpkm")
exprs.set_index("Sample", inplace = True, drop = True)
print(exprs.shape)
exprs.head()

(22665, 399)


,X-1004,X-1008,X-1027,X-1095,X-1119,X-1156,X-1167,X-1169,X-1172,X-1173,...,X-5713,X-5717,X-5727,X-5739,X-5808,X-5959,X-5974,X-5975,X-6030,X-6047
Sample,,,,,,,,,,,,,,,,,,,,,
A1BG,2.75,8.97,0.14,0.13,0.08,17.52,0.18,0.41,0.08,0.00,...,21.24,0.46,18.70,0.00,8.30,14.41,5.09,6.35,0.09,3.22
A1BG-AS1,2.48,3.25,0.00,0.00,0.00,6.52,0.00,0.00,0.00,0.00,...,15.21,0.09,3.42,0.00,8.30,8.36,3.71,4.52,0.00,3.24
A1CF,0.02,0.03,1.30,2.83,2.87,0.72,3.41,0.01,1.84,2.96,...,0.01,0.45,0.00,0.01,0.00,0.00,0.02,0.03,4.63,0.02
A2LD1,4.87,0.81,6.45,4.94,11.07,0.87,3.28,0.32,0.61,7.10,...,1.33,2.51,2.67,3.13,0.44,2.96,0.00,1.99,2.57,1.50
A2M,0.01,71.17,0.00,3.69,0.00,58.16,0.00,0.02,94.12,0.02,...,463.32,0.04,33.35,0.00,254.97,41.52,2.32,0.00,0.00,0.16


In [5]:
# read PCT raw dataset

pct  = pd.read_excel(raw_data_dir+"nm.3954-S2.xlsx", "PCT raw data")
print(pct.shape)
pct.head()

(67445, 8)


,Model,Tumor Type,Treatment,Volume (mm3),body weight (g),Days Post T0,% TVol Difference,% BW Difference
0,X-007,GC,BGJ398,202.3,21.5,0,0.0,0.0
1,X-007,GC,BGJ398,590.3,23.0,4,191.8,7.0
2,X-007,GC,BGJ398,796.3,22.7,7,293.6,5.6
3,X-007,GC,BGJ398,1004.5,23.4,11,396.5,8.8
4,X-007,GC,BKM120,288.8,20.4,0,0.0,0.0


# Mapping of gene symbols to EntrezID using current gene_info file provided by NCBI:


 * Download the unzip the file  
\# wget ftp://ftp.ncbi.nih.gov/gene/DATA/GENE_INFO/Mammalia/Homo_sapiens.gene_info.gz
\# gunzip Homo_sapiens.gene_info.gz;
 * Specify *hgnc_file* variable in this notebook
 * Mapping strategy
 
1). Unknown genes and genes belonging to organisms other than H.sapiens were excluded.

2). First each query symbol was tried to match with any of current "Symbol" directly. If the query symbol mapped to one of current symbol but has no Gene ID, the query symbol was marked as not mapped. 

3). If the query symbol matches none of current symbols, we tried to match it with one of "Synonyms". Genes matched no synonym as well as matched ambiguous synonyms correponding more than one Gene ID were condiered not mapped. 

4). At this point, many of not recognized symbols had LOCXXXXXXXXX format. 
According to the documntation provided by NCBI: "Symbols beginning with LOC. When a published symbol is not available, and orthologs have not yet been determined, Gene will provide a symbol that is constructed as 'LOC' + the GeneID. This is not retained when a replacement symbol has been identified, although queries by the LOC term are still supported. In other words, a record with the symbol LOC12345 is equivalent to GeneID = 12345. So if the symbol changes, the record can still be retrieved on the web using LOC12345 as a query, or from any file using GeneID = 12345" e.g. :

    - LOC100093631 -> 100093631 
    - LOC100129726 -> 100129726
    - etc. 

Therefore all genes started with LOC were converted to Gene IDs removing "LOC" from query term. If resulting Gene ID matched none of current Gene IDs the symbol was considered not mapped. 

4). Several pairs of query gene symbols matched current symbol and synonym of the same Entrez gene ID, e.g. AGAP8 and AGAP4, ANXA8L1 and ANXA8L2, etc.
Expressions of these genes were summarized because such genes were merged to a single gene in newer assembly versions.

In [6]:
# read HGNC file

hgnc_file = root_dir + "HGNC/HGNC.txt"

hgnc = pd.read_csv(hgnc_file, sep = "\t", index_col = 0)
print(hgnc.shape, len(set(hgnc.index.values)))
print(hgnc.columns)
hgnc.head()

(48763, 9) 48763
Index(['Approved symbol', 'Approved name', 'Status', 'Previous symbols',
       'Alias symbols', 'Chromosome', 'Accession numbers', 'RefSeq IDs',
       'NCBI Gene ID'],
      dtype='object')


,Approved symbol,Approved name,Status,Previous symbols,Alias symbols,Chromosome,Accession numbers,RefSeq IDs,NCBI Gene ID
HGNC ID,,,,,,,,,
HGNC:5,A1BG,alpha-1-B glycoprotein,Approved,NaN,NaN,19q13.43,NaN,NM_130786,1.0
HGNC:37133,A1BG-AS1,A1BG antisense RNA 1,Approved,"NCRNA00181, A1BGAS, A1BG-AS",FLJ23569,19q13.43,BC040926,NR_015380,503538.0
HGNC:24086,A1CF,APOBEC1 complementation factor,Approved,NaN,"ACF, ASP, ACF64, ACF65, APOBEC1CF",10q11.23,AF271790,NM_014576,29974.0
HGNC:6,A1S9T,"symbol withdrawn, see [HGNC:12469](/data/gene-...",Symbol Withdrawn,NaN,NaN,NaN,NaN,NaN,NaN
HGNC:7,A2M,alpha-2-macroglobulin,Approved,NaN,"FWP007, S863-7, CPAMD5",12p13.31,"BX647329, X68728, M11313",NM_000014,2.0


In [7]:
# filter status with approved gene symbols

approved = hgnc.loc[hgnc["Status"] == "Approved",:]
hgnc_prev = expand(approved[["Previous symbols","NCBI Gene ID"]], column = "Previous symbols", sep = ", ")
hgnc_prev = parse_mapping_table(hgnc_prev, "Previous symbols", "NCBI Gene ID")

2212 rows with both Previous symbols and NCBI Gene ID empty
Ok: no duplicated pairs detected
29026 rows with empty Previous symbols were excluded
196 Previous symbols ids mapped to no NCBI Gene ID
85 Previous symbols mapped to multiple NCBI Gene ID
5229 different Previous symbols mapped to the same NCBI Gene ID
9783 Previous symbols can be mapped directly to NCBI Gene ID


In [8]:
# find gene symbols with aliases

hgnc_syn = expand(approved[["Alias symbols","NCBI Gene ID"]], column = "Alias symbols", sep = ", ")
hgnc_syn = parse_mapping_table(hgnc_syn, "Alias symbols", "NCBI Gene ID")

1899 rows with both Alias symbols and NCBI Gene ID empty
Ok: no duplicated pairs detected
19505 rows with empty Alias symbols were excluded
689 Alias symbols ids mapped to no NCBI Gene ID
1033 Alias symbols mapped to multiple NCBI Gene ID
29209 different Alias symbols mapped to the same NCBI Gene ID
11274 Alias symbols can be mapped directly to NCBI Gene ID


In [9]:
# map gene symbols to gene id

NCBI = pd.read_csv(root_dir + "Homo_sapiens.gene_info/Homo_sapiens.gene_info.txt", sep = "\t")
NCBI.columns
NCBI = NCBI[["#tax_id","GeneID","Symbol","Synonyms","type_of_gene"]]
NCBI = NCBI.loc[NCBI["#tax_id"] == 9606]
NCBI = NCBI.loc[NCBI["type_of_gene"] != "unknown"]
ncbi_symbols = parse_mapping_table(NCBI, "Symbol", "GeneID")

Ok: no empty rows detected
Ok: no duplicated pairs detected
Ok: All Symbol rows are not empty.
Ok: All Symbol are mapped to GeneID
12 Symbol mapped to multiple GeneID
Ok: All GeneID are unique
140871 Symbol can be mapped directly to GeneID


In [10]:
# map synonyms to gene id

ncbi_synonyms = expand(NCBI[["Synonyms","GeneID"]], column = "Synonyms", sep = "|")
ncbi_synonyms = parse_mapping_table(ncbi_synonyms, "Synonyms", "GeneID")

Ok: no empty rows detected
Ok: no duplicated pairs detected
Ok: All Synonyms rows are not empty.
Ok: All Synonyms are mapped to GeneID
3376 Synonyms mapped to multiple GeneID
53208 different Synonyms mapped to the same GeneID
11014 Synonyms can be mapped directly to GeneID


In [11]:
# show mappend and unmapped gene ids

exprss = apply_mappers(exprs, ncbi_symbols, ncbi_synonyms, verbose = True, handle_duplicates = "sum")
df = pd.DataFrame.from_records(exprss[0])
df.set_index(exprss[0].index.values, inplace= True)
exprs = df
exprs.head()

Mapped: 22416 
	directly via main_mapper 19626 
	via alternative mapper 548 
	via one of multiple synonyms in alternative mapper 1426 
	LOC 816 
Unmapped: 249 
	recognized symbols without Entrez ID 0 
	multiple query_ids map to the same target_id 0 
	query_ids map to multiple target_ids in the main mapper 0 
	query_ids map to multiple target_ids in the alternative mapper 80 
	LOC not found in Entrez 34 
	Not found at all: 135


,X-1004,X-1008,X-1027,X-1095,X-1119,X-1156,X-1167,X-1169,X-1172,X-1173,...,X-5713,X-5717,X-5727,X-5739,X-5808,X-5959,X-5974,X-5975,X-6030,X-6047
1,2.75,8.97,0.14,0.13,0.08,17.52,0.18,0.41,0.08,0.00,...,21.24,0.46,18.70,0.00,8.30,14.41,5.09,6.35,0.09,3.22
2,0.01,71.17,0.00,3.69,0.00,58.16,0.00,0.02,94.12,0.02,...,463.32,0.04,33.35,0.00,254.97,41.52,2.32,0.00,0.00,0.16
3,0.00,0.01,0.00,0.00,0.00,0.10,0.00,0.00,0.02,0.00,...,0.03,0.10,0.01,0.02,0.01,0.00,0.00,0.01,0.00,0.02
9,1.47,0.74,5.64,1.98,9.98,3.71,12.41,2.57,1.38,4.77,...,1.19,1.93,0.93,1.27,1.99,3.34,0.27,1.64,11.12,1.60
10,0.00,0.00,0.99,3.06,9.65,0.00,2.31,0.00,0.11,4.48,...,0.02,0.00,0.00,0.00,0.02,0.00,0.03,0.16,1.92,0.04


### FPKM to TPM conversion
Let $X_i$ is a number of fragments mapped to a transcript, $N$ is a total number of fragments sequensed (and assigned to any transcript) and $\widetilde{l_i}$  is an effective length of a transcript (i.e. how many fragments with average length $\mu_{frag}$ can generate a transcript of length $l_i$: $\widetilde{l_i} = l_i - \mu_{frag}+1$)

*FPKM* - fragments per kilobase of exon (i.e. effective length) per million reads mapped

$FPKM_i = \frac{X_i}{\frac{$\widetilde{l_i}}{10^3}*\frac{N}{10^6}} = \frac{X_i}{l_iN}*10^9 $

*TPM* - transcripts per million of transcripts.

In turn, $\frac{X_i}{\widetilde{l_i}}$ is estimated number of transcripts 

$TPM_i = \frac{\frac{X_i}{\widetilde{l_i}}}{\sum_j{\frac{X_j}{\widetilde{l_j}}}}*10^6 $


### how to convert FPKM to TPM
Divide both numerator and denominator by $N$ and mutiply by $10^9$:

$TPM_i = \frac{\frac{X_i}{N\widetilde{l_i}}*10^9}{\sum_j{\frac{X_j}{N\widetilde{l_j}}*10^9}}*10^6 = \frac{FPKM_i}{\sum_j{FPKM_j}} * 10^6$

Sources:
 - What the FPKM? https://haroldpimentel.wordpress.com/2014/05/08/what-the-fpkm-a-review-rna-seq-expression-units/
 - https://www.biostars.org/p/160989/
 - Lior Pachter https://arxiv.org/pdf/1104.3889.pdf


In [12]:
##  FPKM convert to log2(TPM+1)

sum_fpkm = exprs.apply(sum, axis = 0)
sum_fpkm.head()

X-1004    387513.24
X-1008    488175.21
X-1027    722266.30
X-1095    448322.63
X-1119    608236.65
dtype: float64

In [13]:
# convert FPKM to TPM

tpm = exprs / sum_fpkm * 1000000 + 1
tpm.head()

,X-1004,X-1008,X-1027,X-1095,X-1119,X-1156,X-1167,X-1169,X-1172,X-1173,...,X-5713,X-5717,X-5727,X-5739,X-5808,X-5959,X-5974,X-5975,X-6030,X-6047
1,8.096532,19.374550,1.193834,1.289970,1.131528,33.732665,1.364690,1.780637,1.165838,1.000000,...,44.914047,1.791006,35.027009,1.000000,17.015718,28.605711,11.796351,12.305588,1.170204,7.070138
2,1.025806,146.787821,1.000000,9.230680,1.000000,109.660492,1.000000,1.038080,196.108029,1.030471,...,958.921680,1.068783,61.684532,1.000000,492.991273,80.541231,5.920930,1.000000,1.000000,1.301622
3,1.000000,1.020484,1.000000,1.000000,1.000000,1.186830,1.000000,1.000000,1.041459,1.000000,...,1.062025,1.171958,1.018196,1.041289,1.019296,1.000000,1.000000,1.017804,1.000000,1.037703
9,4.793419,2.515849,8.808754,5.416462,17.408087,7.931403,26.143367,5.893263,3.860700,8.267369,...,3.460344,4.318785,2.692252,3.621875,4.839913,7.398548,1.572694,3.919868,22.029635,4.016218
10,1.000000,1.000000,2.370686,7.825442,16.865535,1.000000,5.680192,1.000000,1.228027,7.825538,...,1.041350,1.000000,1.000000,1.000000,1.038592,1.000000,1.063633,1.284865,4.631016,1.075405


In [14]:
# take logarithm and save the file

tpm = tpm.applymap(np.log2)
tpm.to_csv(preprocessing_data_dir + "exprs/PDX.FPKM2TPMplus1log2.Expr.tsv",sep="\t")
print(tpm.shape)
tpm.head()

(22377, 399)


,X-1004,X-1008,X-1027,X-1095,X-1119,X-1156,X-1167,X-1169,X-1172,X-1173,...,X-5713,X-5717,X-5727,X-5739,X-5808,X-5959,X-5974,X-5975,X-6030,X-6047
1,3.017304,4.276091,0.255603,0.367337,0.178272,5.076074,0.448574,0.832394,0.221367,0.000000,...,5.489095,0.840770,5.130396,0.000000,4.088796,4.838231,3.560269,3.621242,0.226760,2.821738
2,0.036757,7.197588,0.000000,3.206437,0.000000,6.776900,0.000000,0.053917,7.615505,0.043304,...,9.905269,0.095969,5.946837,0.000000,8.945418,6.331656,2.565824,0.000000,0.000000,0.380310
3,0.000000,0.029254,0.000000,0.000000,0.000000,0.247114,0.000000,0.000000,0.058607,0.000000,...,0.086818,0.228921,0.026016,0.058371,0.027573,0.000000,0.000000,0.025460,0.000000,0.053393
9,2.261055,1.331045,3.138938,2.437351,4.121686,2.987576,4.708373,2.559067,1.948862,3.047428,...,1.790916,2.110625,1.428814,1.856737,2.274981,2.887242,0.653238,1.970805,4.461374,2.005837
10,0.000000,0.000000,1.245304,2.968172,4.076006,0.000000,2.505940,0.000000,0.296342,2.968190,...,0.058455,0.000000,0.000000,0.000000,0.054629,0.000000,0.089000,0.361617,2.211329,0.104881


# TCGA 

* from http://gdac.broadinstitute.org/runs/stddata__2016_01_18/data/ download RSEM files, "scaled estimate" per gene. 

* RSEM Scaled estimate is an aboundance of a transcript divided by sum of aboundance over all the transcripts.  Therefore $TPM_i=ScaledEstimate_i*10^6$

* Resulted TPM were log2-transformed

In [16]:
# save TCGA expression datasets as to cohorts

f_ext = ".rnaseqv2__illuminahiseq_rnaseqv2__unc_edu__Level_3__RSEM_genes__data.data.txt"
for fpath in os.listdir(tcga_tmp_dir):
    if fpath.startswith("gdac.broadinstitute.org") and not fpath.endswith(".tar.gz"):
        fpath = fpath.replace(".tar","")
        cohort = fpath.split("_")[1].replace(".Merge","")
        #print(fpath, cohort)
        fname = cohort + f_ext
        try:
            exprs = pd.read_csv(tcga_tmp_dir+"/"+fpath+"/"+fname,sep="\t",index_col=0, low_memory=False)
            # drop "gene_id" and keep only "scaled_estimate" columns
            exprs = exprs.loc[:,exprs.T.loc[exprs.T["gene_id"] == "scaled_estimate", :].index]
            exprs = exprs.loc[exprs.index != "gene_id",]
            exprs.rename(index = lambda x: int(x.split("|")[1]),
                         columns = lambda x: x.replace(".1",""), inplace = True)
            exprs.index.name = "ENTREZID"
            # convert scaled_extimates to log2(TPM+1)
            exprs = exprs.applymap(lambda x: np.log2(float(x) * 1000000 + 1))
            exprs = exprs.sort_index()
            exprs.to_csv(preprocessing_data_dir + "exprs/" +"TCGA-" + cohort + "_exprs.RSEMscaled_est2TPMplus1log2.tsv",
                         sep = "\t")
            print(cohort, exprs.shape)
            
        except:
            print(cohort, "No expression data")

THYM (20531, 122)
THCA (20531, 568)
MESO (20531, 87)
UCS (20531, 57)
UVM (20531, 80)
PRAD (20531, 550)
STES (20531, 646)
LUAD (20531, 576)
LUSC (20531, 552)
SKCM (20531, 473)
PAAD (20531, 183)
OV (20531, 307)
STAD (20531, 450)
LIHC (20531, 423)
UCEC (20531, 381)
SARC (20531, 265)
PCPG (20531, 187)
READ (20531, 72)
TGCT (20531, 156)
ACC (20531, 79)
CHOL (20531, 45)
COAD (20531, 191)
CESC (20531, 309)
DLBC (20531, 48)
BLCA (20531, 427)
COADREAD (20531, 263)
ESCA (20531, 196)
GBM (20531, 171)
KICH (20531, 91)
HNSC (20531, 566)
BRCA (20531, 1212)
GBMLGG (20531, 701)
LAML (20531, 173)
KIRP (20531, 323)
KIPAN (20531, 1020)
KIRC (20531, 606)
LGG (20531, 530)
LIHC (20531, 423)
MESO (20531, 87)
PAAD (20531, 183)
OV (20531, 307)
LUSC (20531, 552)
LUAD (20531, 576)
PCPG (20531, 187)
READ (20531, 72)
SARC (20531, 265)
PRAD (20531, 550)
STAD (20531, 450)
SKCM (20531, 473)
TGCT (20531, 156)
THYM (20531, 122)
UCS (20531, 57)
STES (20531, 646)
UCEC (20531, 381)
THCA (20531, 568)
UVM (20531, 80)
